In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import torch
from tqdm.auto import tqdm
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
def block(input_dim,out_dim):
    return nn.Sequential(
    nn.Linear(input_dim,out_dim),
    nn.BatchNorm1d(out_dim),
    nn.ReLU()
    )
class Model(nn.Module):
    def __init__(self,input_dim=784,num_classes=10,hidden_dim=128):
        super(Model,self).__init__()
        self.net=nn.Sequential(
        block(input_dim,hidden_dim*2),
        block(hidden_dim*2,num_classes)
        )
    def forward(self,x):
        return self.net(x)

In [3]:
model=Model().to(device)
x=torch.randn(64,784)
a=model(x).shape

In [4]:
a

torch.Size([64, 10])

In [5]:
input_size=784
num_classes=10
num_epochs=1
lr=.001
batch_size=64

In [6]:
train_dataset = datasets.MNIST(root='.',train = True,transform=transforms.ToTensor(),download=False)

In [7]:
train_data=DataLoader(train_dataset,batch_size=batch_size,shuffle=True)

In [8]:
test_dataset = datasets.MNIST(root='.',train=False,transform=transforms.ToTensor,download=False)

In [9]:
test_data = DataLoader(test_dataset,batch_size=batch_size,shuffle=True)

In [10]:
model=Model(input_size,num_classes).to(device)

In [11]:
criterion=nn.CrossEntropyLoss()
optim=optim.Adam(model.parameters(),lr=lr)

In [12]:
for epoch in range(num_epochs):
    for data,labels in tqdm(train_data):
        data=data.to(device)
        labels=labels.to(device)
        data=data.reshape(data.shape[0],-1)
        preds=model(data)
        loss=criterion(preds,labels)
        optim.zero_grad()
        loss.backward()
        optim.step() 

In [13]:
def acc(loader,model):
    num_correct=0
    num_samples=0
    model.eval()
    with torch.no_grad():
        for x,y in loader:
            x=x.to(device)
            y=y.to(device)
            x=x.reshape(x.shape[0],-1)
            preds=model(x) #10 classes 
            value,index=preds.max(1)
            num_correct+=(index==y).sum()
            num_samples+=index.size(0)
        acc=num_correct/num_samples
        print(acc)

In [14]:
acc(train_data,model)

tensor(0.9765)
